In [4]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [5]:
model = gp.Model("VRPLTT")

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
Q_min = 140
Q_max = 300
n_levels = 4
load_levels = np.array([(0.5+i)*(Q_max-Q_min)/n_levels for i in range(n_levels)])
upper = load_levels+(Q_max-Q_min)/(n_levels*2)
lower = load_levels-(Q_max-Q_min)/(n_levels*2)

In [7]:
load_levels

array([ 20.,  60., 100., 140.])

In [8]:
upper

array([ 40.,  80., 120., 160.])

In [9]:

def vel(m, h, P):
    ### aer. resistence
    Cd = 1.18
    A = 0.83 #m^2
    rho = 1.18 #kg/m^3

    c_Fd = rho*Cd*A*0.5

    ### rolling resistance
    Cr = 0.01
    g = 9.81 #m/s^2
    
    eff = 0.95
    
    c1 = (m*g* ( Cr*np.cos( np.arctan(h) ) + np.sin( np.arctan(h) ) ))/eff
    c3 = c_Fd/eff
    
    coefs = [c3, 0, c1, -P]
    #print(f"{np.real(np.roots(coefs))=}")
    v = np.max(np.real(3.6*np.roots(coefs)))
    return min(v,30)
    #return 3.6*np.roots(coefs)
    

def time_matrix(data_matrix, load_levels,P):
    N = len(data_matrix.index)
    elevation = data_matrix.elevation.to_numpy()
    d_ij_mat = data_matrix.iloc[:, 8:].to_numpy()
    t_ij_mat = np.zeros((N, N, n_levels))
    for i in range(N):
        for j in range(N):
            if i!=j:
                d_ij = d_ij_mat[i, j]
                h = (elevation[j] - elevation[i])/d_ij
                for l in range(n_levels):
                    m = load_levels[l]
                    t_ij_mat[i, j, l] =60*d_ij/vel(m, h, P)
    return np.round(t_ij_mat,4)

In [50]:
df = pd.read_csv("./instances/small/Fukuoka_01.csv")
P = 500 # Watt

t=time_matrix(df,load_levels,P=P)

In [51]:
from pprint import pprint
print(t[3,4])
print(t[4,3])


[1.1577 3.4176 5.6932 7.9698]
[0.332 0.332 0.332 0.332]


In [76]:
def extract_min(Q,distance):
    minimo=float("inf") #highest python value
    nome=""
    for q in Q: #find the smallest
        if distance[q] <= minimo:
            nome=q
            minimo=distance[q]
    Q.remove(nome) #remove the correct node from the que
    return nome # tell me wich node is removed

def W(u,v,weight_matrix):
    for i in adj[u]:
        if i.end==v:
            return i.weight
        
def dijkstra(source,weight_matrix,l=0):
    distance=dict() #dict for distance from source
    parent=dict() #dict for parentnes of each node
    nodes=np.array([i for i in range(len(weight_matrix))]) #list of current node to iterate
    for node in nodes:
        distance[node]=float("inf") 
        parent[node]=None
    distance[source]=0
    S=[]
    Q={node for node in nodes} #set used ad que, coupled with distance dict
    while len(Q)>0:
        u=extract_min(Q,distance) #extract the min from Q,using distance dict
        S.append(u) #ultimated nodes
        for v,w in enumerate(weight_matrix[u]): #for each node v, outgoing from u, with weight w
            if distance[v] > distance[u] + w[l]: #relax phase
                distance[v]= distance[u] + w[l]
                parent[v]=u
        #print(f"{u = }")
        #print(f"{distance = }")
    return distance,parent #return distances from source and each node parent !
            
dist,parent = dijkstra(0,t)
pprint(dist)
pprint(parent)

{0: 0,
 1: 0.38,
 2: 1.4763000000000002,
 3: 2.5689,
 4: 3.3916,
 5: 3.6915999999999998,
 6: 4.0356,
 7: 5.315099999999999,
 8: 5.2126,
 9: 6.8701,
 10: 6.2101,
 11: 5.761,
 12: 3.2565999999999997,
 13: 3.1090999999999998,
 14: 4.5546,
 15: 6.0489999999999995,
 16: 6.0104999999999995,
 17: 2.3936,
 18: 1.9616,
 19: 1.1192,
 20: 2.1546000000000003}
{0: None,
 1: 0,
 2: 1,
 3: 2,
 4: 18,
 5: 4,
 6: 5,
 7: 13,
 8: 12,
 9: 10,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 12,
 15: 1,
 16: 14,
 17: 18,
 18: 0,
 19: 0,
 20: 19}


In [84]:
for i in t[:,:,2]:
    for j in i:
        print(f"\t{round(j,3)}",end=" ")
    print("\n")

	0.0 	0.38 	7.916 	16.632 	22.683 	27.272 	32.231 	40.478 	45.222 	46.574 	35.213 	32.998 	20.72 	17.824 	29.672 	33.01 	36.477 	12.204 	9.647 	5.455 	10.934 

	0.38 	0.0 	5.346 	11.376 	17.145 	21.98 	27.245 	34.957 	38.728 	40.05 	28.676 	26.467 	14.149 	13.423 	26.86 	27.883 	34.69 	15.19 	14.092 	7.427 	9.652 

	0.696 	0.36 	0.0 	5.372 	11.068 	16.05 	21.533 	28.985 	33.315 	37.129 	26.92 	23.302 	12.428 	11.669 	25.109 	26.128 	32.936 	13.252 	13.365 	0.824 	11.059 

	1.01 	0.674 	0.314 	0.0 	5.693 	10.597 	15.452 	23.596 	28.336 	32.143 	27.367 	18.313 	13.426 	15.552 	27.399 	27.883 	34.584 	1.488 	18.044 	1.138 	0.988 

	1.342 	1.006 	0.646 	0.332 	0.0 	0.3 	0.646 	17.792 	24.486 	32.209 	27.436 	18.381 	19.136 	21.236 	33.05 	33.592 	40.283 	1.82 	1.43 	1.47 	1.32 

	1.642 	1.306 	0.946 	0.632 	4.853 	0.0 	0.344 	12.88 	19.338 	27.809 	23.375 	21.237 	24.292 	26.366 	38.166 	32.01 	39.46 	2.122 	12.792 	1.772 	1.62 

	1.988 	1.65 	1.292 	0.978 	9.844 	4.529 	0.0 	7.046 	13.39 

In [78]:
%%timeit
dist,parent = dijkstra(0,t)


264 µs ± 8.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [83]:
def shortest_path_matrix(t_ij_mat):
    sp_matrix=np.empty_like(t_ij_mat)
    nodes=np.array([i for i in range(t_ij_mat.shape[0])])
    levels=np.array([i for i in range(t_ij_mat.shape[-1])])
    print(levels)
    for source in nodes:
        for l in levels:
            distance,_t=dijkstra(source,t_ij_mat,l)
            sp_matrix[source,:,l]=list(distance.values())
    return sp_matrix
sp = shortest_path_matrix(t)
for i in sp[:,:,1]:
    for j in i:
        print(f"\t{round(j,2)}",end=" ")
    print("\n")
    


[0 1 2 3]
	0.0 	0.38 	3.59 	6.82 	7.22 	7.52 	7.87 	10.64 	10.83 	16.27 	17.59 	15.7 	8.87 	8.44 	10.17 	15.5 	14.47 	6.22 	5.79 	3.28 	6.17 

	0.38 	0.0 	3.21 	6.44 	7.6 	7.9 	8.25 	10.26 	10.45 	15.89 	17.21 	15.32 	8.49 	8.06 	9.79 	15.12 	14.09 	6.6 	6.17 	3.66 	5.79 

	0.7 	0.36 	0.0 	3.23 	6.64 	6.94 	7.29 	9.21 	9.42 	14.86 	16.16 	13.99 	7.46 	7.0 	8.76 	14.09 	13.06 	4.71 	6.49 	0.82 	3.88 

	1.01 	0.67 	0.31 	0.0 	3.42 	3.72 	4.06 	8.29 	9.73 	15.11 	16.43 	10.99 	7.77 	7.32 	9.07 	13.74 	13.37 	1.49 	4.85 	1.14 	0.99 

	1.34 	1.01 	0.65 	0.33 	0.0 	0.3 	0.64 	4.87 	8.68 	14.22 	14.16 	11.03 	8.11 	6.35 	9.4 	14.08 	13.7 	1.82 	1.43 	1.47 	1.32 

	1.64 	1.31 	0.95 	0.63 	2.91 	0.0 	0.34 	4.57 	8.38 	13.92 	13.86 	11.63 	8.41 	7.95 	9.7 	14.38 	14.0 	2.12 	4.34 	1.77 	1.62 

	1.99 	1.65 	1.29 	0.98 	4.4 	2.72 	0.0 	4.23 	8.04 	13.58 	13.52 	11.97 	8.75 	8.3 	9.91 	14.22 	14.2 	2.47 	5.83 	2.12 	1.96 

	2.4 	2.06 	1.7 	1.39 	1.06 	0.75 	0.41 	0.0 	5.29 	10.83 	11.9 	12.02 	9.16